<a href="https://colab.research.google.com/github/Shreeshambav/DeepLearning_training/blob/main/Deeplearning_Sarima_ETS_Model_STATS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings

# Read the data
file_path_merged = 'C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_Cat_with_outliers_replaced.xlsx'
df = pd.read_excel(file_path_merged)

# Drop rows with missing values in the specified columns
df = df.dropna(subset=["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])

# Convert the values in columns "01" to "12" to numeric
df[["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]] = df[["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]].apply(pd.to_numeric, errors="coerce")

# Calculate the sum of values for each row
df["Sum"] = df[["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]].sum(axis=1)

# Filter rows where the sum of values is greater than 0
filtered_data = df[df["Sum"] > 0]

# Group the data by "COUNTRY" and "Year" and calculate the mean of the sum for each group
grouped_data = filtered_data.groupby(["COUNTRY", "Year"])["Sum"].mean()

print(grouped_data)

# # Group the data based on 'MRL_Category'
# grouped_df = df.groupby('MRL_Category')


# Group the data based on the new 'MRL_Category' columns
grouped_df = df.groupby(['MRL_Category_Cat1', 'MRL_Category_Cat2', 'MRL_Category_Cat3', 'MRL_Category_Cat4'])

# Helper function to create a concatenated MRL category string
def get_category_string(row):
    return f"{row['MRL_Category_Cat1']}_{row['MRL_Category_Cat2']}_{row['MRL_Category_Cat3']}_{row['MRL_Category_Cat4']}"

# Define ARIMA model parameters
# order = (5, 1, 0)  # (p, d, q) - ARIMA parameters

# Create an empty DataFrame to store the predictions
predictions_df = pd.DataFrame(columns=["Category", "Year", "Month", "Prediction", "ARIMA_Prediction", "SARIMA_Prediction", "ETS_Prediction"])

# Create an empty DataFrame to store the forecast for year 2022
forecast_df = pd.DataFrame(columns=["Category", "Year", "Month", "ARIMA_Forecast", "SARIMA_Forecast", "ETS_Forecast"])

# Create empty lists to store statistics for ARIMA and Exponential Smoothing separately
arima_stats_list = []
ets_stats_list = []
sarima_stats_list = []

# Filter data for each year (2019, 2020, 2021) and apply ARIMA model for each category
years = [2019, 2020, 2021, 2022]
months = ["{:02d}".format(month) for month in range(1, 13)]

# Helper function to find the last financial year
def find_last_financial_year(years):
    sorted_years = sorted(years, reverse=True)
    for i in range(len(sorted_years) - 1):
        if sorted_years[i] - 1 != sorted_years[i + 1]:
            return sorted_years[i]
    return sorted_years[-1]

# ACF and PACF with upper and lower bound
def plot_acf_pacf(train_data):
    # Calculate ACF and PACF for the training data
    acf_values, confint_acf = acf(train_data, nlags=30, alpha=0.05)
    pacf_values, confint_pacf = pacf(train_data, nlags=30, alpha=0.05)

    # Plot the ACF and PACF with upper and lower bounds
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 1, 1)
    plt.title("Autocorrelation Function (ACF)")
    plt.stem(acf_values, markerfmt='o', linefmt='gray', basefmt='gray', use_line_collection=True)
    plt.axhline(y=-1.96/np.sqrt(len(train_data)), linestyle='--', color='gray')  # Lower bound
    plt.axhline(y=1.96/np.sqrt(len(train_data)), linestyle='--', color='gray')   # Upper bound
    plt.grid(True)

    plt.subplot(2, 1, 2)
    plt.title("Partial Autocorrelation Function (PACF)")
    plt.stem(pacf_values, markerfmt='o', linefmt='gray', basefmt='gray', use_line_collection=True)
    plt.axhline(y=-1.96/np.sqrt(len(train_data)), linestyle='--', color='gray')  # Lower bound
    plt.axhline(y=1.96/np.sqrt(len(train_data)), linestyle='--', color='gray')   # Upper bound
    plt.grid(True)

    plt.tight_layout()
    plt.show()

for category, group_df in grouped_df:
    for year in years:
        year_df = group_df[group_df['Year'] == year]

        # Check if all 12 columns exist in the DataFrame
        data_columns = [str(month).zfill(2) for month in range(1, 13)]
        missing_columns = [col for col in data_columns if col not in year_df.columns]

        if missing_columns:
            print(f"Missing data columns for Category: {category}, Year: {year}: {missing_columns}")
            continue

        data = year_df[data_columns].values.flatten()

        # Handle cases where there might be missing data for some years
        if np.any((data != 0) & pd.notnull(data)):
            # Find last financial year
            last_financial_year = find_last_financial_year(years)

            # Split data into train and test sets
            if year == last_financial_year:
                # Last 12 months as test data
                train_data = data[:-12]
                test_data = data[-12:]
            else:
                # Rest as train data
                train_data = data[:-12]  # Use all available data except the last 12 months
                test_data = data[-12:]  # Use the last 12 months as test data

            # Perform ADF test to check for stationarity
            adf_result = adfuller(train_data)
            p_value = adf_result[1]
            is_stationary = p_value < 0.05
            print(f"Category: {category}, Year: {year} - Time series is Stationary.")

            if not is_stationary:
                print(f"Category: {category}, Year: {year} - Time series is non-stationary.")
                continue

            # Try different ARIMA model orders and methods
            orders_to_try = [(5, 1, 0), (4, 1, 0), (3, 1, 0)]  # Add more orders to try

            model_fit = None
            converged = False
            for order in orders_to_try:
                try:
                    # Try fitting ARIMA model using different orders
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")  # Ignore ConvergenceWarning
                        model = ARIMA(train_data, order=order)
                        model_fit = model.fit()
                        arima_predictions = model_fit.forecast(steps=12)
                    converged = True
                    break  # Break if the model successfully converges
                except Exception as e:
                    print(f"ARIMA model did not converge for Category: {category}, Year: {year}, Order: {order}")
                    print(f"Error message: {str(e)}")
                    continue

            if not converged:
                print(f"ARIMA model did not converge for any combination of orders for Category: {category}, Year: {year}.")
                continue

            # Flatten the arima_predictions to match the shape of test_data
            arima_predictions = arima_predictions.flatten()

            # Fit exponential smoothing model and make predictions for the out-of-sample period (2022)
            ets_model = ExponentialSmoothing(train_data, seasonal='add', seasonal_periods=12)
            ets_fit = ets_model.fit()
            ets_predictions = ets_fit.forecast(steps=12)
            # Flatten the ets_predictions to match the shape of test_data
            ets_predictions = ets_predictions.flatten()

            # Fit SARIMA model and make predictions for the out-of-sample period (2022)
            sarima_model = SARIMAX(train_data, order=order, seasonal_order=(0, 1, 1, 12))
            sarima_fit = sarima_model.fit()
            sarima_predictions = sarima_fit.forecast(steps=12)
            # Flatten the sarima_predictions to match the shape of test_data
            sarima_predictions = sarima_predictions.flatten()

            # ACF and PACF plots
            plot_acf_pacf(train_data)

            # Calculate Mean Squared Error (MSE), Root Mean Squared Error (RMSE), and Mean Absolute Percentage Error (MAPE)
            if np.any(test_data):
                mse_arima = mean_squared_error(test_data.reshape(-1, 1), arima_predictions.reshape(-1, 1))
                rmse_arima = np.sqrt(mse_arima)
                z_scores_test_arima = (test_data - np.mean(test_data)) / np.std(test_data)
                z_scores_pred_arima = (arima_predictions - np.mean(arima_predictions)) / np.std(arima_predictions)

                # Calculate MAPE only for non-zero elements in test_data
                non_zero_indices_arima = test_data != 0
                mape_arima = np.mean(np.abs((test_data[non_zero_indices_arima] - arima_predictions[non_zero_indices_arima]) / test_data[non_zero_indices_arima])) * 100

                print("ARIMA - MSE:", mse_arima, "RMSE:", rmse_arima, "MAPE:", mape_arima, "Z_score:", z_scores_test_arima, "Z_scores:", z_scores_pred_arima)

                mse_ets = mean_squared_error(test_data.reshape(-1, 1), ets_predictions.reshape(-1, 1))
                rmse_ets = np.sqrt(mse_ets)
                z_scores_test_ets = (test_data - np.mean(test_data)) / np.std(test_data)
                z_scores_pred_ets = (ets_predictions - np.mean(ets_predictions)) / np.std(ets_predictions)

                # Calculate MAPE only for non-zero elements in test_data
                non_zero_indices_ets = test_data != 0
                mape_ets = np.mean(np.abs((test_data[non_zero_indices_ets] - ets_predictions[non_zero_indices_ets]) / test_data[non_zero_indices_ets])) * 100

                print("Exponential Smoothing - MSE:", mse_ets, "RMSE:", rmse_ets, "MAPE:", mape_ets, "Z_score:", z_scores_test_ets, "Z_scores:", z_scores_pred_ets)

                mse_sarima = mean_squared_error(test_data.reshape(-1, 1), sarima_predictions.reshape(-1, 1))
                rmse_sarima = np.sqrt(mse_sarima)
                z_scores_test_sarima = (test_data - np.mean(test_data)) / np.std(test_data)
                z_scores_pred_sarima = (sarima_predictions - np.mean(sarima_predictions)) / np.std(sarima_predictions)

                # Calculate MAPE only for non-zero elements in test_data
                non_zero_indices_sarima = test_data != 0
                mape_sarima = np.mean(np.abs((test_data[non_zero_indices_sarima] - sarima_predictions[non_zero_indices_sarima]) / test_data[non_zero_indices_sarima])) * 100

                print("SARIMA - MSE:", mse_sarima, "RMSE:", rmse_sarima, "MAPE:", mape_sarima, "Z_score:", z_scores_test_sarima, "Z_scores:", z_scores_pred_sarima)
            else:
                print(f"No test data available for Category: {category}, Year: {year}")

            # Append the statistics values to the appropriate lists
            arima_stats_list.append({"Category": category, "Year": year, "Model": "ARIMA", "MSE": mse_arima, "RMSE": rmse_arima, "MAPE": mape_arima, "z_scores_t": z_scores_test_arima, "z_scores_p": z_scores_pred_arima})
            ets_stats_list.append({"Category": category, "Year": year, "Model": "Exponential Smoothing", "MSE": mse_ets, "RMSE": rmse_ets, "MAPE": mape_ets, "z_scores_t": z_scores_test_ets, "z_scores_p": z_scores_pred_ets})
            sarima_stats_list.append({"Category": category, "Year": year, "Model": "SARIMA", "MSE": mse_sarima, "RMSE": rmse_sarima, "MAPE": mape_sarima, "z_scores_t": z_scores_test_sarima, "z_scores_p": z_scores_pred_sarima})

            # Store the predictions and forecasts in the respective DataFrames
            for i in range(12):
                month = months[i]
                prediction_row = {"Category": category, "Year": year, "Month": month,
                                  "Prediction": test_data[i], "ARIMA_Prediction": arima_predictions[i],
                                  "SARIMA_Prediction": sarima_predictions[i], "ETS_Prediction": ets_predictions[i]}
                predictions_df = predictions_df.append(prediction_row, ignore_index=True)

            # Forecast for year 2022 (for all 12 months)
            if year == last_financial_year:
                for i in range(12):
                    month = months[i]
                    forecast_row = {"Category": category, "Year": 2022, "Month": month,
                                    "ARIMA_Forecast": arima_predictions[i], "SARIMA_Forecast": sarima_predictions[i], "ETS_Forecast": ets_predictions[i] }
                    forecast_df = forecast_df.append(forecast_row, ignore_index=True)

# Save the predictions and forecast DataFrames to CSV
predictions_df.to_csv("C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Sarima\\predictions.csv", index=False)
forecast_df.to_csv("C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Sarima\\forecast.csv", index=False)

# Save the statistics DataFrames to CSV
arima_stats_df = pd.DataFrame(arima_stats_list)
ets_stats_df = pd.DataFrame(ets_stats_list)
sarima_stats_df = pd.DataFrame(sarima_stats_list)
arima_stats_df.to_csv("C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Sarima\\arima_stats.csv", index=False)
ets_stats_df.to_csv("C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Sarima\\ets_stats.csv", index=False)
sarima_stats_df.to_csv("C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Sarima\\sarima_stats.csv", index=False)
